In [232]:
import sys 
import os 
import re 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from pathlib import Path

sys.path.append("/Users/ngokykhanhthu/Documents/Mental_Health_n_LLMs")
from mentalLLM.config import RAW_DIR, INTERIM_DIR, PROCESSED_DIR, FINAL_DIR, DATASET, DATA_KEYWORD, CHAT_GPT_QUESTION
from chat_gpt_scraper import remove_white_spaces, is_unix_time, convert_unix_time, find_links, scrape_link, retrieve_the_json_portion, extract_json_object, create_csv_files, extract_msg

In [233]:
# load the valid responses only 
df = pd.read_csv("../data/4.analysis/1.0-valid_data.csv")

# get all valid links

In [234]:
df_chats = df.copy()

# extract links for gpt 
get all the columns that have gpt links 
concatenate them vertically 
spits out the unique values 

In [235]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [236]:
columns = [col for col in df.columns if col.endswith("llm_gpt")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

19

In [237]:
gpt_links

['https://chatgpt.com/share/68e80b00-f5b0-8013-a344-6aaccceb8336',
 'https://chatgpt.com/share/68e80ec1-9684-8000-b406-249e306bd728',
 'https://chatgpt.com/share/68f1543e-8a44-8003-9877-578d28b398c4',
 'https://chatgpt.com/share/68f52102-9200-800b-8ba7-579ba8a629dd',
 'https://chatgpt.com/share/68f5321a-2468-8012-8dd8-872b818a6fd0',
 'https://chatgpt.com/share/68f5321a-2468-8012-8dd8-872b818a6fd0',
 'https://chatgpt.com/share/690aaf87-12a0-8009-b6a1-c7cc060bfcdc',
 'https://chatgpt.com/share/690bf9cc-2e0c-800c-8a7e-1c8d36eaaa06',
 'https://chatgpt.com/share/690e5633-3a1c-8013-8a8e-f0fa1e2e299b',
 'https://chatgpt.com/share/690e5697-abf0-8013-a2b5-5bfb6194422b',
 'https://chatgpt.com/share/691433c9-dda0-800c-ad8d-8753093f30b4',
 'https://chatgpt.com/share/690bf9cc-2e0c-800c-8a7e-1c8d36eaaa06',
 'https://chatgpt.com/share/691ce685-b098-8013-9dc6-3e5a094608a3',
 'https://chatgpt.com/share/691cdfcd-c1bc-8012-844f-a7c9b47afa05',
 'https://chatgpt.com/share/691ce022-71d4-8012-93cb-ed7d6bb0ec

# claude 

In [238]:
columns = [col for col in df.columns if col.endswith("llm_claude")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

1

In [239]:
gpt_links

['https://claude.ai/share/470b75ef-24ce-40a3-8190-a3cbda282610']

# cursor 

In [240]:
columns = [col for col in df.columns if col.endswith("llm_cursor_Id")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

0

# copilot 

In [241]:
columns = [col for col in df.columns if col.endswith("llm_copilot_Id")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)



1

In [242]:
gpt_links

['F_2UflSx5ZJr1Bn03']

# deepseek

In [243]:
columns = [col for col in df.columns if col.endswith("llm_deepseek_Id")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

0

# other

In [244]:
columns = [col for col in df.columns if col.endswith("llm_other_Id")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

0

In [245]:
columns = [col for col in df.columns if col.endswith("llm_other_link")]
gpt_links = df_chats[columns].stack().reset_index(drop = True).tolist()
len(gpt_links)

3

In [246]:
gpt_links

['https://gemini.google.com/share/66df5ceb8e2c',
 'https://poe.com/s/W4QPWtDrn9S28eXSYkji',
 'https://gemini.google.com/share/f56d0ed62e68']

# Chat purposes

In [247]:
df["chat_purpose"]

0                 1,8,10
1                1,2,3,4
2                    1,4
3                    1,4
4                    NaN
5                  2,4,8
6                  1,4,8
7                    6,8
8              1,2,4,8,9
9       1,2,3,4,6,8,9,10
10               1,2,4,6
11                   NaN
12              2,3,4,10
13                   NaN
14                     1
15               1,2,4,7
16          1,3,4,6,8,10
17    1,2,3,4,5,6,7,8,10
18             2,3,4,6,9
19                   4,8
Name: chat_purpose, dtype: object

In [248]:
def map_occupations(value, custom_map):
    if pd.isna(value):
        return value
    codes = str(value).split(',')
    mapped = [custom_map.get(code.strip(), code.strip()) for code in codes]
    return ', '.join(mapped)


In [249]:
occupation_map = {"1": "Generate code", 
            "2": "Explain concepts/syntax", 
            "3": "Providing alternative code solutions to tasks", 
            "4": "Debug code", 
            "5": "Generate documentation/standardise syntax", 
            "6": "Brainstorm code implementations", 
            "7": "Generate test cases", 
            "8": "Find the solution to a coding task", 
            "9": "Provide insights on design patterns", 
            "10": "Optimise code"}

# data.replace({"prgming_occupation": occupation_map}, inplace=True)

df["chat_purpose"] = df["chat_purpose"].apply(
    lambda x: map_occupations(x, occupation_map)
)

answers = ["Generate code", "Explain concepts/syntax", "Providing alternative code solutions to tasks", "Debug code", 
            "Generate documentation/standardise syntax", "Brainstorm code implementations", "Generate test cases",
            "Find the solution to a coding task", "Provide insights on design patterns", "Optimise code"]


result = pd.DataFrame({
    'Chat Purposes': answers,
    'Count': [df["chat_purpose"].str.contains(re.escape(answer), na=False).sum() for answer in answers]
})

result

,Chat Purposes,Count
0,Generate code,12
1,Explain concepts/syntax,9
2,Providing alternative code solutions to tasks,6
3,Debug code,14
4,Generate documentation/standardise syntax,1
5,Brainstorm code implementations,6
6,Generate test cases,2
7,Find the solution to a coding task,9
8,Provide insights on design patterns,3
9,Optimise code,5


In [250]:
df["chat_llmcode_used_1"].max()

np.float64(96.0)

# get the chat feelings 

In [251]:
# select columns 
feelings = [col for col in df.columns if col.startswith("chat_feelings")]

text = df[feelings].iloc[:, -1:]
non_text = df[feelings].iloc[:, :-1]

In [252]:
series = non_text.mean(axis=0)
series.round(2)

chat_feelings_1    3.59
chat_feelings_2    3.71
chat_feelings_3    3.59
chat_feelings_4    3.71
chat_feelings_5    2.65
chat_feelings_6    3.65
chat_feelings_7    1.94
chat_feelings_8    3.18
dtype: float64

In [253]:
non_text.count()

chat_feelings_1    17
chat_feelings_2    17
chat_feelings_3    17
chat_feelings_4    17
chat_feelings_5    17
chat_feelings_6    17
chat_feelings_7    17
chat_feelings_8    17
dtype: int64

In [254]:
text.value_counts()

chat_feelings_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# validated pmh scale 

In [286]:
pmh_cols = [col for col in df.columns if col.startswith("pmh_validated")]
# sum_series = df[pmh_cols].dropna()
series = sum_series.sum(axis = 1)
series

0     21.0
1     14.0
2     29.0
3     21.0
6     30.0
7     21.0
8     17.0
9     26.0
10    11.0
12    12.0
14    19.0
15    26.0
16    27.0
17    17.0
18    23.0
19    32.0
dtype: float64

In [256]:
mean = series.mean()
std = series.std()
mean

np.float64(21.625)

In [257]:
# standardised series  
standardised_series = (series - mean)/std
standardised_series.size
standardised_series


# iqr 
quantile_1 = series.quantile(.25)
quantile_3 = series.quantile(.75)
iqr = quantile_3 - quantile_1
standardised_iqr = iqr*1.5/std

# median 
standardised_median = (series.median() - mean)/std
standardised_series[(standardised_series >= standardised_median + standardised_iqr) | (standardised_series <= standardised_median - standardised_iqr)]


Series([], dtype: float64)

In [258]:
list = df.index[df[pmh_cols].sum(axis = 1) == 0].tolist()
df.iloc[list, df.columns.get_loc("pmh_validated_1"): df.columns.get_loc("pmh_validated_9")]

,pmh_validated_1,pmh_validated_2,pmh_validated_3,pmh_validated_4,pmh_validated_5,pmh_validated_6,pmh_validated_7,pmh_validated_8
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [270]:
series[series <= mean].index

Index([0, 1, 3, 7, 8, 10, 12, 14, 17], dtype='int64')

In [ ]:
# given a list of indices, fetch me the whole data frame 
# and then examine chats they have sent in, their points on how they felt when they code, scores on the self efficacy scale 



In [263]:
series.min()

np.float64(11.0)

In [264]:
series.max()

np.float64(32.0)

In [265]:
4*8

32

In [276]:
series = series - 1
series.mean()

np.float64(19.625)

In [309]:
series.min()

np.float64(11.0)

In [310]:
series[series == 11]

10    11.0
dtype: float64

In [278]:
series[series < series.mean()].size

9

In [281]:
series[series > 29.3]

19    30.0
dtype: float64

In [283]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [284]:
df.iloc[19]

Unnamed: 0                                 62
StartDate                 2025-12-05 13:07:29
EndDate                   2025-12-05 13:15:51
Status                                      0
Progress                                  100
                                 ...         
dem_gender_3_TEXT                         NaN
dem_institution                           2.0
dem_institution_7_TEXT                    NaN
Logistics_1                               1.0
Logistics_2                               2.0
Name: 19, Length: 223, dtype: object

# get the depressed people as well 

In [279]:
32*33/36

29.333333333333332

In [272]:
(33.0*9)/8

37.125

In [273]:
4*8

32

llm_copilot_Id
llm_deepseek_Id
llm_other_Id
llm_other_link
llm_gpt

In [302]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_claude")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

[]

In [303]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_copilot_Id")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

[]

In [304]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_deepseek_Id")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

[]

In [305]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_other_Id")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

[]

In [306]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_other_link")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

[]

In [307]:
subset= df.loc[[4, 5, 11, 13, 15], :]
columns = [col for col in subset.columns if col.endswith("llm_gpt")]
gpt_links = subset[columns].stack().tolist()
len(gpt_links)
gpt_links

['https://chatgpt.com/share/68f5321a-2468-8012-8dd8-872b818a6fd0',
 'https://chatgpt.com/share/68f5321a-2468-8012-8dd8-872b818a6fd0',
 'https://chatgpt.com/share/690bf9cc-2e0c-800c-8a7e-1c8d36eaaa06',
 'https://chatgpt.com/share/691ce685-b098-8013-9dc6-3e5a094608a3']

In [308]:
subset

,Unnamed: 0,StartDate,EndDate,Status,Progress,Finished,RecordedDate,DistributionChannel,Q_RecaptchaScore,consent_1,eligibility_1,eligibility_1.1,eligibility_2,general_num_chat,1_llm_type,1_llm_gpt,1_llm_claude,1_llm_cursor_Id,1_llm_cursor_Name,1_llm_cursor_Size,1_llm_cursor_Type,1_llm_copilot_Id,1_llm_copilot_Name,1_llm_copilot_Size,1_llm_copilot_Type,1_llm_deepseek_Id,1_llm_deepseek_Name,1_llm_deepseek_Size,1_llm_deepseek_Type,1_llm_other_name,1_llm_other_link,1_llm_other_file_Id,1_llm_other_file_Name,1_llm_other_file_Size,1_llm_other_file_Type,1_llm_project,1_llm_project_link,1_llm_project_file_Id,1_llm_project_file_Name,1_llm_project_file_Size,1_llm_project_file_Type,2_llm_type,2_llm_gpt,2_llm_claude,2_llm_cursor_Id,2_llm_cursor_Name,2_llm_cursor_Size,2_llm_cursor_Type,2_llm_copilot_Id,2_llm_copilot_Name,2_llm_copilot_Size,2_llm_copilot_Type,2_llm_deepseek_Id,2_llm_deepseek_Name,2_llm_deepseek_Size,2_llm_deepseek_Type,2_llm_other_name,2_llm_other_link,2_llm_other_file_Id,2_llm_other_file_Name,2_llm_other_file_Size,2_llm_other_file_Type,2_llm_project,2_llm_project_link,2_llm_project_file_Id,2_llm_project_file_Name,2_llm_project_file_Size,2_llm_project_file_Type,3_llm_type,3_llm_gpt,3_llm_claude,3_llm_cursor_Id,3_llm_cursor_Name,3_llm_cursor_Size,3_llm_cursor_Type,3_llm_copilot_Id,3_llm_copilot_Name,3_llm_copilot_Size,3_llm_copilot_Type,3_llm_deepseek_Id,3_llm_deepseek_Name,3_llm_deepseek_Size,3_llm_deepseek_Type,3_llm_other_name,3_llm_other_link,3_llm_other_file_Id,3_llm_other_file_Name,3_llm_other_file_Size,3_llm_other_file_Type,3_llm_project,3_llm_project_link,3_llm_project_file_Id,3_llm_project_file_Name,3_llm_project_file_Size,3_llm_project_file_Type,4_llm_type,4_llm_gpt,4_llm_claude,4_llm_cursor_Id,4_llm_cursor_Name,4_llm_cursor_Size,4_llm_cursor_Type,4_llm_copilot_Id,4_llm_copilot_Name,4_llm_copilot_Size,4_llm_copilot_Type,4_llm_deepseek_Id,4_llm_deepseek_Name,4_llm_deepseek_Size,4_llm_deepseek_Type,4_llm_other_name,4_llm_other_link,4_llm_other_file_Id,4_llm_other_file_Name,4_llm_other_file_Size,4_llm_other_file_Type,4_llm_project,4_llm_project_link,4_llm_project_file_Id,4_llm_project_file_Name,4_llm_project_file_Size,4_llm_project_file_Type,5_llm_type,5_llm_gpt,5_llm_claude,5_llm_cursor_Id,5_llm_cursor_Name,5_llm_cursor_Size,5_llm_cursor_Type,5_llm_copilot_Id,5_llm_copilot_Name,5_llm_copilot_Size,5_llm_copilot_Type,5_llm_deepseek_Id,5_llm_deepseek_Name,5_llm_deepseek_Size,5_llm_deepseek_Type,5_llm_other_name,5_llm_other_link,5_llm_other_file_Id,5_llm_other_file_Name,5_llm_other_file_Size,5_llm_other_file_Type,5_llm_project,5_llm_project_link,5_llm_project_file_Id,5_llm_project_file_Name,5_llm_project_file_Size,5_llm_project_file_Type,chat_purpose,chat_llmcode_used_1,chat_feelings_1,chat_feelings_2,chat_feelings_3,chat_feelings_4,chat_feelings_5,chat_feelings_6,chat_feelings_7,chat_feelings_8,chat_feelings_text,efficacy_1_1,efficacy_1_2,efficacy_1_3,efficacy_1_4,efficacy_1_5,efficacy_1_6,efficacy_2_1,efficacy_2_2,efficacy_2_3,efficacy_2_4,efficacy_2_5,efficacy_3_1,efficacy_3_2,efficacy_3_3,efficacy_3_4,efficacy_3_5,efficacy_3_6,efficacy_4_1,efficacy_4_2,efficacy_4_3,prgming_years_1,prgming_occupation,prgming_freq_code,prgming_freq_llm,prgming_degree,prgming_major,prgming_dsa,prgming_swe,prgming_swe_3_TEXT,prgming_job_title,prgming_job_code_hz,prgming_experience,prgming_tool_comfort_1,prgming_tool_comfort_2,prgming_tool_comfort_3,prgming_tool_comfort_4,prgming_tool_comfort_5,prgming_tool_comfort_6,pmh_name,pmh_name_7_TEXT,pmh_validated_1,pmh_validated_2,pmh_validated_3,pmh_validated_4,pmh_validated_5,pmh_validated_6,pmh_validated_7,pmh_validated_8,pmh_validated_9,neurotype_name,neurotype_name_5_TEXT,neurotype_validated_1,neurotype_validated_2,neurotype_validated_3,neurotype_validated_4,neurotype_validated_5,neurotype_validated_6,dem_gender,dem_gender_3_TEXT,dem_institution,dem_institution_7_TEXT,Logistics_1,Logistics_2
4,17,2025-10-19 14:43:04,2025-10-19 14:50:18,0,39,0,2025-10-26 14:50:19,anonymous,0.4,1,21,2,2,1,1,htt

In [314]:
col_ind = [df.columns.get_loc(col) for col in feelings]
df.iloc[10, col_ind]

chat_feelings_1       4.0
chat_feelings_2       4.0
chat_feelings_3       5.0
chat_feelings_4       5.0
chat_feelings_5       5.0
chat_feelings_6       4.0
chat_feelings_7       2.0
chat_feelings_8       5.0
chat_feelings_text    NaN
Name: 10, dtype: object

In [319]:
text

,chat_feelings_text
0,NaN
1,"I'm not sure what the last question means! If it means that I rarely ask LLMs to generate code for two problems that look similar, then I strongly agree."
2,What is more
3,NaN
4,NaN
5,I feel good. I feel like i am doing more in less amount of time
6,"I do not feel confident in coding because I have failed multiple computer science classes in the past, despite also doing well in some computer science classes. I do find it very helpful to use LLMs in coding because there are some cases when completing coding homework, where I feel the teacher expects me to be able to perform a task that has not been previously demonstrated in class. In these cases, LLMs are incredibly helpful in filling in those gaps in knowledge where I would previously try to use websites like geeksforgeeks and try to find users with similar problems to me. Still, the users on these websites often solved them in very complex ways that I would have a hard time mimicking, even when comparing their solutions to documentation, creating more errors for myself and resulting in some of my poor grades in the past. A problem that arises when I use LLMs is that VSCode Copilot immediately fills in the next line of code according to the instructions provided by the teacher, which, while helpful, creates an issue of making it harder for me personally to use my knowledge to add on to the code where I can. Ultimately, I think LLMs are helpful to me when coding, but my use of them does come from insecurity about being able to do it myself."
7,Sometimes frustration because LLM can overthink and overcomplicate a lot. So sometimes it actually saves me time--I needed to spend extra time debugging later.
8,NaN
9,"It depends on what type of coding session im doing. If it is a field I'm confident in then it would feel fine to use AI to help as I can do the code myself anyway, it just helps making everything faster. If it is a field I'm clueless in then it is pretty bad. I have to rely on AI for everything. And at the end of the day you feel like you only learn the design pattern and nothing else (no syntax, debug, etc). I have become pretty resentful towards coding as a whole when I have to rely too much on AI as it sucks all the fun out of coding. But in the day and age when everyone uses AI, you have to adapt I guess."


In [320]:
subset = df.drop(index=[4, 5, 11, 13, 15])
subset[feelings]

,chat_feelings_1,chat_feelings_2,chat_feelings_3,chat_feelings_4,chat_feelings_5,chat_feelings_6,chat_feelings_7,chat_feelings_8,chat_feelings_text
0,5.0,5.0,4.0,2.0,2.0,5.0,2.0,4.0,NaN
1,4.0,4.0,5.0,5.0,2.0,4.0,1.0,3.0,"I'm not sure what the last question means! If it means that I rarely ask LLMs to generate code for two problems that look similar, then I strongly agree."
2,5.0,4.0,4.0,5.0,1.0,5.0,5.0,2.0,What is more
3,4.0,4.0,2.0,5.0,2.0,4.0,1.0,4.0,NaN
6,2.0,2.0,5.0,1.0,2.0,1.0,1.0,5.0,"I do not feel confident in coding because I have failed multiple computer science classes in the past, despite also doing well in some computer science classes. I do find it very helpful to use LLMs in coding because there are some cases when completing coding homework, where I feel the teacher expects me to be able to perform a task that has not been previously demonstrated in class. In these cases, LLMs are incredibly helpful in filling in those gaps in knowledge where I would previously try to use websites like geeksforgeeks and try to find users with similar problems to me. Still, the users on these websites often solved them in very complex ways that I would have a hard time mimicking, even when comparing their solutions to documentation, creating more errors for myself and resulting in some of my poor grades in the past. A problem that arises when I use LLMs is that VSCode Copilot immediately fills in the next line of code according to the instructions provided by the teacher, which, while helpful, creates an issue of making it harder for me personally to use my knowledge to add on to the code where I can. Ultimately, I think LLMs are helpful to me when coding, but my use of them does come from insecurity about being able to do it myself."
7,4.0,5.0,2.0,4.0,2.0,5.0,2.0,4.0,Sometimes frustration because LLM can overthink and overcomplicate a lot. So sometimes it actually saves me time--I needed to spend extra time debugging later.
8,2.0,4.0,3.0,4.0,2.0,4.0,1.0,4.0,NaN
9,1.0,1.0,4.0,5.0,1.0,1.0,5.0,2.0,"It depends on what type of coding session im doing. If it is a field I'm confident in then it would feel fine to use AI to help as I can do the code myself anyway, it just helps making everything faster. If it is a field I'm clueless in then it is pretty bad. I have to rely on AI for everything. And at the end of the day you feel like you only learn the design pattern and nothing else (no syntax, debug, etc). I have become pretty resentful towards coding as a whole when I have to rely too much on AI as it sucks all the fun out of coding. But in the day and age when everyone uses AI, you have to adapt I guess."
10,4.0,4.0,5.0,5.0,5.0,4.0,2.0,5.0,NaN
12,4.0,4.0,5.0,5.0,4.0,4.0,1.0,2.0,"I felt guilty for resorting to using it, but most of the time, I do so with the intention of learning---for instance, how I can make my code (the code that I wrote myself) more efficient, or search for alternative code to compare my solutions and learn new concepts/techniques. I have never copied-pasted entire solutions derived by LLMs, and in cases where I did incorporate snippets generated by it into my program, I did it with the full understanding of what the snippet is doing differently."
